# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([28, 81,  3, 53, 56, 30, 61, 41,  7, 62, 62, 62, 31,  3, 53, 53, 12,
       41, 19,  3, 26, 77, 65, 77, 30, 43, 41,  3, 61, 30, 41,  3, 65, 65,
       41,  3, 65, 77, 48, 30, 52, 41, 30, 39, 30, 61, 12, 41, 68, 80, 81,
        3, 53, 53, 12, 41, 19,  3, 26, 77, 65, 12, 41, 77, 43, 41, 68, 80,
       81,  3, 53, 53, 12, 41, 77, 80, 41, 77, 56, 43, 41, 67, 21, 80, 62,
       21,  3, 12, 18, 62, 62,  4, 39, 30, 61, 12, 56, 81, 77, 80], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:characters_per_batch * n_batches]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n: n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, 0:-1] = x[:, 1:]
        y[:, -1] = x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
b = get_batches(encoded, 2, 32)
x, y = next(b)
print("x: \n", x)
print("y: \n", y)

x: 
 [[28 81  3 53 56 30 61 41  7 62 62 62 31  3 53 53 12 41 19  3 26 77 65 77
  30 43 41  3 61 30 41  3]
 [41 56 30  3 61  1 43 56  3 77 80 30 57 60 41 53 77 56 77 19 68 65 60 41
  43 21 30 30 56 41 19  3]]
y: 
 [[81  3 53 56 30 61 41  7 62 62 62 31  3 53 53 12 41 19  3 26 77 65 77 30
  43 41  3 61 30 41  3 28]
 [56 30  3 61  1 43 56  3 77 80 30 57 60 41 53 77 56 77 19 68 65 60 41 43
  21 30 30 56 41 19  3 41]]


In [9]:
print("x's shape: \n", x.shape)
print("y's shape: \n", y.shape)

x's shape: 
 (2, 32)
y's shape: 
 (2, 32)


In [10]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[28 81  3 53 56 30 61 41  7 62]
 [41 56 30  3 61  1 43 56  3 77]]

y
 [[81  3 53 56 30 61 41  7 62 62]
 [56 30  3 61  1 43 56  3 77 80]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [11]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name="inputs")
    targets = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name="targets")
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [12]:


def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm_cells = [tf.contrib.rnn.BasicLSTMCell(lstm_size) for _ in range(num_layers)]
    
    # Add dropout to the cell outputs
    lstm_cells = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) for lstm in lstm_cells]
    
    # Stack up multiple LSTM layers, for deep learning

    cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [13]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat((lstm_output), axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, shape=(-1, in_size))
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(
            tf.truncated_normal((in_size, out_size), stddev=0.1),
            name="softmax_w"
        )
        softmax_b = tf.Variable(
            tf.zeros(out_size),
            name="softmax_b"
        )

    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name="predications")
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [14]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, depth=num_classes)
    y_reshaped = tf.reshape(y_one_hot, shape=logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [15]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [16]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
        
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, depth=num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        lstm_outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(lstm_outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)


## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [22]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 256         # Size of hidden layers in LSTMs
num_layers = 2         # Number of LSTM layers
learning_rate = 0.005    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [23]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        batches = get_batches(encoded, batch_size, num_steps)
        for x, y in batches:
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4168...  2.7553 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.0440...  2.8731 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.8536...  2.1222 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 3.6617...  2.0287 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 3.5697...  2.0195 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.4796...  1.9183 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.3868...  1.9083 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.3118...  1.9457 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.2891...  2.0432 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.2658...  2.0174 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.2200...  2.5222 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.2174...  2.0447 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 2.7803...  2.1497 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 2.7521...  2.2895 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 2.7485...  2.0542 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 2.7517...  1.9620 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 2.7045...  1.9533 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 2.7260...  1.9333 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 2.7274...  2.0592 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 2.6743...  2.1531 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 2.6993...  1.9431 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 2.7151...  1.8982 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 2.6870...  1.9588 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 2.6701...  2.0813 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.4007...  2.0647 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.3863...  2.4917 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.3822...  2.0895 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.3856...  1.7564 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.3917...  1.7335 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.3619...  1.6807 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.3701...  1.7061 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.3538...  1.7318 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.3770...  1.7551 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.4030...  1.6551 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.3656...  1.8113 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.3595...  1.8593 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 303...  Training loss: 2.1529...  2.7924 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.1750...  2.2616 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.1664...  2.6682 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.1716...  2.5663 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.1788...  2.5770 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.1468...  2.7801 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.1666...  2.5286 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.1620...  2.4954 sec/batch
Epoch: 2/20...  Training Step: 311...  Training loss: 2.1560...  3.1552 sec/batch
Epoch: 2/20...  Training Step: 312...  Training loss: 2.1563...  3.8565 sec/batch
Epoch: 2/20...  Training Step: 313...  Training loss: 2.1522...  2.6858 sec/batch
Epoch: 2/20...  Training Step: 314...  Training loss: 2.1078...  2.3150 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 403...  Training loss: 2.0864...  2.1813 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.0906...  2.3273 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.1171...  2.0569 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.0795...  1.8914 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.0716...  2.1455 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.0663...  2.1870 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 2.0887...  2.4975 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 2.1107...  2.4742 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 2.0834...  1.8256 sec/batch
Epoch: 3/20...  Training Step: 412...  Training loss: 2.0560...  1.7357 sec/batch
Epoch: 3/20...  Training Step: 413...  Training loss: 2.0853...  1.8573 sec/batch
Epoch: 3/20...  Training Step: 414...  Training loss: 2.1116...  1.8864 sec/batch
Epoch: 3/20...  

...

Epoch: 18/20...  Training Step: 3559...  Training loss: 1.4323...  1.6807 sec/batch
Epoch: 18/20...  Training Step: 3560...  Training loss: 1.4139...  1.7045 sec/batch
Epoch: 18/20...  Training Step: 3561...  Training loss: 1.4026...  1.6980 sec/batch
Epoch: 18/20...  Training Step: 3562...  Training loss: 1.4433...  1.7386 sec/batch
Epoch: 18/20...  Training Step: 3563...  Training loss: 1.4187...  1.6864 sec/batch
Epoch: 18/20...  Training Step: 3564...  Training loss: 1.4174...  1.6839 sec/batch
Epoch: 19/20...  Training Step: 3565...  Training loss: 1.4903...  1.6873 sec/batch
Epoch: 19/20...  Training Step: 3566...  Training loss: 1.4375...  1.6894 sec/batch
Epoch: 19/20...  Training Step: 3567...  Training loss: 1.4244...  1.7162 sec/batch
Epoch: 19/20...  Training Step: 3568...  Training loss: 1.4457...  1.7350 sec/batch
Epoch: 19/20...  Training Step: 3569...  Training loss: 1.4266...  2.1263 sec/batch
Epoch: 19/20...  Training Step: 3570...  Training loss: 1.3939...  2.0341 se

Epoch: 19/20...  Training Step: 3657...  Training loss: 1.4011...  2.8421 sec/batch
Epoch: 19/20...  Training Step: 3658...  Training loss: 1.4084...  2.5675 sec/batch
Epoch: 19/20...  Training Step: 3659...  Training loss: 1.4058...  2.4624 sec/batch
Epoch: 19/20...  Training Step: 3660...  Training loss: 1.4431...  2.7109 sec/batch
Epoch: 19/20...  Training Step: 3661...  Training loss: 1.4348...  1.9110 sec/batch
Epoch: 19/20...  Training Step: 3662...  Training loss: 1.3944...  2.3879 sec/batch
Epoch: 19/20...  Training Step: 3663...  Training loss: 1.4141...  2.5810 sec/batch
Epoch: 19/20...  Training Step: 3664...  Training loss: 1.3905...  2.6500 sec/batch
Epoch: 19/20...  Training Step: 3665...  Training loss: 1.4174...  2.6237 sec/batch
Epoch: 19/20...  Training Step: 3666...  Training loss: 1.4163...  2.2601 sec/batch
Epoch: 19/20...  Training Step: 3667...  Training loss: 1.4265...  2.1797 sec/batch
Epoch: 19/20...  Training Step: 3668...  Training loss: 1.4120...  1.8723 se

Epoch: 19/20...  Training Step: 3755...  Training loss: 1.4118...  2.6603 sec/batch
Epoch: 19/20...  Training Step: 3756...  Training loss: 1.4153...  2.9076 sec/batch
Epoch: 19/20...  Training Step: 3757...  Training loss: 1.4144...  2.9987 sec/batch
Epoch: 19/20...  Training Step: 3758...  Training loss: 1.4004...  2.5030 sec/batch
Epoch: 19/20...  Training Step: 3759...  Training loss: 1.3914...  2.4672 sec/batch
Epoch: 19/20...  Training Step: 3760...  Training loss: 1.4318...  3.2229 sec/batch
Epoch: 19/20...  Training Step: 3761...  Training loss: 1.4082...  3.2972 sec/batch
Epoch: 19/20...  Training Step: 3762...  Training loss: 1.4078...  3.1994 sec/batch
Epoch: 20/20...  Training Step: 3763...  Training loss: 1.4809...  2.1882 sec/batch
Epoch: 20/20...  Training Step: 3764...  Training loss: 1.4415...  2.9337 sec/batch
Epoch: 20/20...  Training Step: 3765...  Training loss: 1.4198...  2.5686 sec/batch
Epoch: 20/20...  Training Step: 3766...  Training loss: 1.4354...  2.2115 se

Epoch: 20/20...  Training Step: 3853...  Training loss: 1.3885...  1.8123 sec/batch
Epoch: 20/20...  Training Step: 3854...  Training loss: 1.4090...  1.7453 sec/batch
Epoch: 20/20...  Training Step: 3855...  Training loss: 1.3996...  1.8994 sec/batch
Epoch: 20/20...  Training Step: 3856...  Training loss: 1.3980...  1.7643 sec/batch
Epoch: 20/20...  Training Step: 3857...  Training loss: 1.3997...  1.7818 sec/batch
Epoch: 20/20...  Training Step: 3858...  Training loss: 1.4339...  1.7429 sec/batch
Epoch: 20/20...  Training Step: 3859...  Training loss: 1.4278...  1.7334 sec/batch
Epoch: 20/20...  Training Step: 3860...  Training loss: 1.3886...  1.8079 sec/batch
Epoch: 20/20...  Training Step: 3861...  Training loss: 1.3970...  1.7796 sec/batch
Epoch: 20/20...  Training Step: 3862...  Training loss: 1.3942...  1.7677 sec/batch
Epoch: 20/20...  Training Step: 3863...  Training loss: 1.4098...  1.7509 sec/batch
Epoch: 20/20...  Training Step: 3864...  Training loss: 1.4063...  1.7485 se

Epoch: 20/20...  Training Step: 3951...  Training loss: 1.4048...  1.7865 sec/batch
Epoch: 20/20...  Training Step: 3952...  Training loss: 1.4043...  1.7926 sec/batch
Epoch: 20/20...  Training Step: 3953...  Training loss: 1.3936...  1.7656 sec/batch
Epoch: 20/20...  Training Step: 3954...  Training loss: 1.3937...  1.8102 sec/batch
Epoch: 20/20...  Training Step: 3955...  Training loss: 1.4170...  1.7557 sec/batch
Epoch: 20/20...  Training Step: 3956...  Training loss: 1.3978...  1.7302 sec/batch
Epoch: 20/20...  Training Step: 3957...  Training loss: 1.3907...  1.8302 sec/batch
Epoch: 20/20...  Training Step: 3958...  Training loss: 1.4215...  1.7526 sec/batch
Epoch: 20/20...  Training Step: 3959...  Training loss: 1.4095...  1.7529 sec/batch
Epoch: 20/20...  Training Step: 3960...  Training loss: 1.4039...  1.7566 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [24]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i3960_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l256.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l256.ckpt"
all_model_checkpoint_pa

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [25]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [26]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [27]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i3960_l256.ckpt'

In [34]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3960_l256.ckpt
Farshas to be high to time of his sister's one went into the particuled,
and hostelistly that the tervation of
and to could be as in a stout women, and the solution of which she was an only should she had seeing the day, but the clints, he was a mother and
had been to see, and to be hards in the day it was a contenticle, that it had
say his
wance. He was the deacte with
his feeling of surrings the sole clear, while he had been that he had always seemed of her face of his heart. There wanted into healding at the sight of a commost and hand of the times to say his wife at
her words, but that he would say, said as he could not spire that the place who, all at a sound
and he was to be stone of the carriage of them.

"You can't have an incloudly. I can't
see you would be as to till your face of my step, I'll can she was took in a minuses they think with his husband, but anythy terribly it," he said, and with an
impression

In [30]:
checkpoint = 'checkpoints/i200_l256.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l256.ckpt
Farer on witt, the to wering the ser the the sis he hit has and so seer had sor the teedens hare alind he ald and wamith on the han so tee toull weunt whe that and to sat hat his the and wang to chite the
san hor tine she that ond hit
shes sith whot of of
and the ar the seed the terinsed, hat and, ang hor, af to te sos the she se at hans
sind o mere soudsering at the wan wan was alle as the athe thetered on hor ther ane wers ond the shr widit his he wersith an wer ter thot that ande sat hat and so wid sont ase soung, and wing what and his he torle sith wer and whes shis ans, wos har ale wam son the sortithin hot he wong wor hond wam has of the ang the was to wat he were as that thas the ald ans thitt and tot af atert the seante and that her
sous as had his thame whos
wis he ther sis to cortirit he
wat hane he wes tish an andens of on hos thin sase and
hers onde had an he te a thet tane to hat his of hared the wis her 

In [32]:
checkpoint = 'checkpoints/i600_l256.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l256.ckpt
Farer wasted, and soor of stised, and wat tole a mating and her she
what the taint which answer and had she concencow was
thought thinked him,
ablooking the carting of her. She seeting of the stiress of it.

And
stor stomened," seid the hingry.



Chisters, she
said, and his sain to the hand of the carrion as tertal of the
couthel of the wand that the telled to and he was bround it.... I dour to the sore her sometions to the cantant the said all the peetings all the stord the come, the sers to his her her at the sole, work to that she had sears
wealds thit wis so the chanding the telling her head at she his was the
preasing and
havand of sond that when so alone.

"Yes, whether said of hard of it
stores the selled his
went
of the
wassing that
him there stent
on the sear had soonen hard the sather it what he say the carrity this were the seening of and at him a dacition the with the
hushing a thind. He sendicted of he h

In [35]:
checkpoint = 'checkpoints/i1200_l256.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l256.ckpt
Faring the seamed.

"I as he's
not it
of thome and why he could not have stell a way that him to have her at the
mompere in
the morried the showed in the thishiss. There to
the
place to her way that though he
wanted.

He did not said to her,
that he had break of she would not be a caming the thought, and who spoke a thout of so were all her
all how sone him out to at
the poseress,
and a starring on
her how the sole and tranded. She heard along in the warding the promartions was so this husperially his steping is, and that the seepes or the convirted as and, the
more....

The sands and his shele worse at the sant and the
mat the concead in a teater had been the candle talked of
her sead. Stepan Arkadyevitch should this she doed it was that to the prosain of the countell to the more of which
all the to sort that the carried, had all his has to the caming. Stepan Arkadyevitch, he was a man with her was seet, and the sho